# ZeusDB Vector Store

ZeusDB is a high-performance, Rust-powered vector database with enterprise features like quantization and persistence.

This notebook covers how to get started with the ZeusDB Vector Store to efficiently use ZeusDB with LangChain.

---

## Setup

Install the ZeusDB LangChain integration package:

In [ ]:
%pip install -qU langchain-zeusdb

---

## 1. Initialization

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_zeusdb import ZeusDBVectorStore
from zeusdb import VectorDatabase

# Initialize embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create ZeusDB index
vdb = VectorDatabase()
index = vdb.create(
    index_type="hnsw",
    dim=1536,
    space="cosine"
)

# Create vector store
vector_store = ZeusDBVectorStore(
    zeusdb_index=index,
    embedding=embeddings
)

---

## 2. Manage Vector Store

### 2.1 Add items to vector store

In [ ]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="ZeusDB is a high-performance vector database",
    metadata={"source": "https://docs.zeusdb.com"}
)

document_2 = Document(
    page_content="Product Quantization reduces memory usage significantly",
    metadata={"source": "https://docs.zeusdb.com"}
)

document_3 = Document(
    page_content="ZeusDB integrates seamlessly with LangChain",
    metadata={"source": "https://docs.zeusdb.com"}
)

documents = [document_1, document_2, document_3]

vector_store.add_documents(documents=documents, ids=["1", "2", "3"])

### 2.2 Update items in vector store

In [ ]:
updated_document = Document(
    page_content=(
        "ZeusDB now supports advanced Product Quantization "
        "with 4x-256x compression"
    ),
    metadata={"source": "https://docs.zeusdb.com", "updated": True}
)

vector_store.add_documents([updated_document], ids=["1"])

### 2.3 Delete items from vector store

In [ ]:
vector_store.delete(ids=["3"])

---

## 3. Query Vector Store

### 3.1 Query directly

Performing a simple similarity search:

In [ ]:
results = vector_store.similarity_search(
    query="high performance database", 
    k=2
)

for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

If you want to execute a similarity search and receive the corresponding scores:

In [ ]:
results = vector_store.similarity_search_with_score(
    query="memory optimization", 
    k=2
)

for doc, score in results:
    print(f"* [SIM={score:.3f}] {doc.page_content} [{doc.metadata}]")

### 3.2 Query by turning into retriever

You can also transform the vector store into a retriever for easier usage in your chains:

In [ ]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 2}
)

retriever.invoke("vector database features")

---

## 4. ZeusDB-Specific Features

### 4.1 Memory-Efficient Setup with Product Quantization

For large datasets, use Product Quantization to reduce memory usage:

In [ ]:
# Create memory-optimized vector store
quantization_config = {
    'type': 'pq',
    'subvectors': 8,
    'bits': 8,
    'training_size': 10000
}

vdb_quantized = VectorDatabase()
quantized_index = vdb_quantized.create(
    index_type="hnsw",
    dim=1536,
    quantization_config=quantization_config
)

quantized_vector_store = ZeusDBVectorStore(
    zeusdb_index=quantized_index,
    embedding=embeddings
)

print(f"Created quantized store: {quantized_index.info()}")

### 4.2 Persistence

Save and load your vector store to disk:

In [ ]:
# Save the vector store
vector_store.save_index("my_zeusdb_index.zdb")

# Load the vector store
loaded_store = ZeusDBVectorStore.load_index(
    path="my_zeusdb_index.zdb",
    embedding=embeddings
)

print(f"Loaded store with {loaded_store.get_vector_count()} vectors")

---

## Usage for Retrieval-Augmented Generation (RAG)

For guides on how to use this vector store for retrieval-augmented generation (RAG), see the following sections:

- [How-to: Question and answer with RAG](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [Retrieval conceptual docs](https://python.langchain.com/docs/concepts/retrieval/)